change: using the city names given by uszips.csv

Using files from shared google drive vs local files.

Make sure the shared folder 'dataSci TEAM project' is located in your 'My Drive' (you can create a shortcut)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests

## Loading in data

### Zipcode Data

In [2]:
# load in zipcode data
zipcode_df = pd.read_csv('datasets/uszips.csv')
zipcode_to_city = zipcode_df[['zip', 'city', 'state_id']]
zipcode_to_city = zipcode_to_city.rename(columns={'state_id': 'state', 'zip': 'zipcode'})
zipcode_to_city.head(3)

,zipcode,city,state
0,601,Adjuntas,PR
1,602,Aguada,PR
2,603,Aguadilla,PR


### Listings Data

In [3]:
listing = pd.read_csv('datasets/listings.csv')
listing_selected_columns = listing.drop(columns=['review_scores_value','weekly_price'])

In [4]:
# since there are only 46 missing value in city, compare to the 60k rows of dataset, it's ok to drop 'city' has na, 
listing_selected_columns = listing_selected_columns[listing_selected_columns['city'].notna()]
listing_selected_columns = listing_selected_columns[listing_selected_columns['zipcode'].notna()]
listing_selected_columns = listing_selected_columns[['zipcode', 'accommodates', 'availability_30']]
listing_selected_columns["total_space_available"]=listing_selected_columns["accommodates"]*listing_selected_columns["availability_30"]

listing_selected_columns.head()


,zipcode,accommodates,availability_30,total_space_available
0,10464,2.0,24,48.0
1,10464,4.0,30,120.0
2,10464,4.0,30,120.0
3,10464,3.0,8,24.0
4,10464,4.0,17,68.0


In [5]:
# clean the zipcodes
listing_selected_columns["zipcode"] = listing_selected_columns["zipcode"].str[:5]
listing_selected_columns["zipcode"] = listing_selected_columns["zipcode"].replace(
    [" ", "1m"], 
    ["20017", "10025"])
listing_selected_columns["zipcode"] = listing_selected_columns["zipcode"].apply(lambda x: int(float(x)))

listing_selected_columns.head()

,zipcode,accommodates,availability_30,total_space_available
0,10464,2.0,24,48.0
1,10464,4.0,30,120.0
2,10464,4.0,30,120.0
3,10464,3.0,8,24.0
4,10464,4.0,17,68.0


In [6]:
# attach the associated city and state for each zipcode
temp_clean_listing = pd.merge(listing_selected_columns, zipcode_to_city, on="zipcode", how="inner")
temp_clean_listing.head()

,zipcode,accommodates,availability_30,total_space_available,city,state
0,10464,2.0,24,48.0,Bronx,NY
1,10464,4.0,30,120.0,Bronx,NY
2,10464,4.0,30,120.0,Bronx,NY
3,10464,3.0,8,24.0,Bronx,NY
4,10464,4.0,17,68.0,Bronx,NY


In [7]:
temp_clean_listing = temp_clean_listing[['zipcode', 'city', 'state', 'total_space_available']]
temp_clean_listing.head()

,zipcode,city,state,total_space_available
0,10464,Bronx,NY,48.0
1,10464,Bronx,NY,120.0
2,10464,Bronx,NY,120.0
3,10464,Bronx,NY,24.0
4,10464,Bronx,NY,68.0


In [8]:
# find the total_space_available for each city
clean_listing = temp_clean_listing.pivot_table(index=['city', 'state'], values='total_space_available', aggfunc='sum').sort_values(by='total_space_available', ascending=False)
clean_listing.reset_index(inplace = True)

# count how many total Airbnb rental listing by location
count_listing = temp_clean_listing.pivot_table(index=['city','state'], values='total_space_available', aggfunc = 'count').sort_values(by = 'total_space_available',ascending=False)
count_listing.reset_index(inplace = True)
count_listing.rename(columns={'total_space_available': 'listing_count'}, inplace=True)

#add to clean listing dataset
clean_listing = clean_listing.merge(count_listing, on=['city','state'], how='left')
clean_listing.head()

,city,state,total_space_available,listing_count
0,New York,NY,346785.0,18903
1,Brooklyn,NY,336442.0,16527
2,Washington,DC,257492.0,7628
3,Chicago,IL,209510.0,5130
4,Denver,CO,73543.0,2421


In [9]:
clean_listing.listing_count.sum()

58861

In [10]:
clean_listing.shape

(93, 4)

### Demographics Dataset:

In [11]:
demographics = pd.read_csv("datasets/demographics.csv")

# convert these values to NaNs
remove_set = {"-", "#VALUE", "#VALUE!", "(X)", "2,500-", "N"}

#TODO: what value should we put for 250,000+ (appears in median household income)
def string_to_float(x):
    if (x == "250,000+"):
        return float("250000")
    return float(x) if x not in remove_set else None

columns = ["$10,000-$14,999", "$15,000-$24,999", "$25,000-$34,999",
           "$35,000-$49,999", "$50,000-$64,999", "$65,000-$74,999",
           "$75,000-$99,999", "$100,000_or_more", "median_household_income",
           "mean_household_income"]

for column in columns:
    demographics[column] = demographics[column].apply(string_to_float)

# drop the rows with NaN in columns
for column in columns:
    demographics = demographics.drop(demographics[demographics[column].isnull()].index)



In [12]:
# merge demographics and clean_zipcode_df on the zipcode column
clean_demographics = pd.merge(demographics, zipcode_to_city, on="zipcode", how="inner")

# sum across all age groups to find total population
clean_demographics["total_population"] = clean_demographics.iloc[:, 2:14].sum(axis=1)
clean_demographics

# take selected columns only:
clean_demographics = clean_demographics[['zipcode', 'city', 'state', 'total_population','mean_household_income','median_household_income']]
clean_demographics.head(3)


,zipcode,city,state,total_population,mean_household_income,median_household_income
0,601,Adjuntas,PR,17669,20349.0,10816.0
1,602,Aguada,PR,39637,23282.0,16079.0
2,603,Aguadilla,PR,51661,26820.0,16804.0


In [13]:
# find the aggregate demographics for each city
clean_demographics = clean_demographics.pivot_table(index=['city','state'], 
                                       values=['total_population', 'mean_household_income', 'median_household_income'], 
                                       aggfunc = {
                                           'total_population': np.sum,
                                           'mean_household_income': np.mean,
                                           'median_household_income': np.mean})

clean_demographics.reset_index(inplace = True)
clean_demographics.head(3)


,city,state,mean_household_income,median_household_income,total_population
0,Aaronsburg,PA,57441.0,53000.0,1019
1,Abbeville,AL,45482.0,33944.0,6257
2,Abbeville,GA,40817.0,29200.0,4635



### ECON Dataset:

In [14]:
original_econ_df = pd.read_csv("datasets/econ_state.csv", index_col ="state")

# Extract columns that are 2015
# original_econ_df.columns[original_econ_df.columns.str.contains("2015")]

econ_df_2015 = original_econ_df[['2015Q1_gdp', '2015Q2_gdp', '2015Q3_gdp', '2015Q4_gdp',
       '2015Q1_personal_income', '2015Q2_personal_income',
       '2015Q3_personal_income', '2015Q4_personal_income', '2015/01_ur',
       '2015/02_ur', '2015/03_ur', '2015/04_ur', '2015/05_ur', '2015/06_ur',
       '2015/07_ur', '2015/08_ur', '2015/09_ur', '2015/10_ur', '2015/11_ur',
       '2015/12_ur']]
# Checking if there is nan values
econ_df_2015.isna().sum()

2015Q1_gdp                0
2015Q2_gdp                0
2015Q3_gdp                0
2015Q4_gdp                0
2015Q1_personal_income    0
2015Q2_personal_income    0
2015Q3_personal_income    0
2015Q4_personal_income    0
2015/01_ur                0
2015/02_ur                0
2015/03_ur                0
2015/04_ur                0
2015/05_ur                0
2015/06_ur                0
2015/07_ur                0
2015/08_ur                0
2015/09_ur                0
2015/10_ur                0
2015/11_ur                0
2015/12_ur                0
dtype: int64

### Get average GDP and unemployment rate for each state

In [15]:
econ_df_2015["annual_GDP"] = econ_df_2015.loc[:, ['2015Q1_gdp', '2015Q2_gdp', '2015Q3_gdp', '2015Q4_gdp']].sum(axis=1)
econ_df_2015["personal_income_average"] = econ_df_2015.loc[:, ['2015Q1_personal_income', '2015Q2_personal_income', '2015Q3_personal_income', '2015Q4_personal_income']].mean(axis=1)
econ_df_2015["ur_average"] = econ_df_2015.loc[:, ['2015/01_ur',
       '2015/02_ur', '2015/03_ur', '2015/04_ur', '2015/05_ur', '2015/06_ur',
       '2015/07_ur', '2015/08_ur', '2015/09_ur', '2015/10_ur', '2015/11_ur']].mean(axis=1)
econ_df_2015.head()

C:\Users\zheng\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\zheng\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\zheng\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,2015Q1_gdp,2015Q2_gdp,2015Q3_gdp,2015Q4_gdp,2015Q1_personal_income,2015Q2_personal_income,2015Q3_personal_income,2015Q4_personal_income,2015/01_ur,2015/02_ur,...,2015/06_ur,2015/07_ur,2015/08_ur,2015/09_ur,2015/10_ur,2015/11_ur,2015/12_ur,annual_GDP,personal_income_average,ur_average
state,,,,,,,,,,,,,,,,,,,,,
AL,197479,199018,200942,201185,182587916,183994420,185488848,187068484,6.4,6.4,...,6.4,6.4,6.4,6.5,6.5,6.5,6.5,798624,1.847849e+08,6.427273
AK,53364,53915,52493,51216,41339300,41527308,41460148,41516228,6.0,6.0,...,6.1,6.1,6.1,6.1,6.1,6.1,6.1,210988,4.146075e+07,6.072727
AZ,286045,289694,292221,295652,262555548,266428700,268797224,271663056,5.6,5.5,...,5.2,5.0,4.9,4.7,4.6,4.4,4.3,1163612,2.673611e+08,5.100000
AR,116930,118707,119997,119994,112268628,113582196,114692240,115151092,6.3,6.2,...,5.9,5.9,5.8,5.8,5.7,5.7,5.6,475628,1.139235e+08,5.945455
CA,2438787,2475826,2492851,2517930,2054273268,2097203792,2118913364,2144287468,6.8,6.6,...,6.2,6.1,6.0,5.9,5.8,5.8,5.7,9925394,2.103669e+09,6.218182


In [16]:
econ_df_2015.drop(['2015Q1_gdp', '2015Q2_gdp', '2015Q3_gdp', '2015Q4_gdp',       '2015Q1_personal_income', '2015Q2_personal_income',       '2015Q3_personal_income', '2015Q4_personal_income', '2015/01_ur',       '2015/02_ur', '2015/03_ur', '2015/04_ur', '2015/05_ur', '2015/06_ur',       '2015/07_ur', '2015/08_ur', '2015/09_ur', '2015/10_ur', '2015/11_ur',       '2015/12_ur'], axis=1, inplace=True)

C:\Users\zheng\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [17]:
clean_econ= econ_df_2015.reset_index()
clean_econ.head(4)

,state,annual_GDP,personal_income_average,ur_average
0,AL,798624,184784917.0,6.427273
1,AK,210988,41460746.0,6.072727
2,AZ,1163612,267361132.0,5.100000
3,AR,475628,113923539.0,5.945455


### Real_estate Dataset and extract 2015 data

In [18]:
original_estate_df = pd.read_csv("datasets/real_estate.csv.gz", compression='gzip')

# Extract columns that are 2015
# original_estate_df.columns[original_estate_df.columns.str.contains("2015")]

estate_df = original_estate_df[['type', 'zipcode', 'city', 'state', 'metro', 'county', 'size_rank', '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12']]

# Check for any nan values
estate_df.isna().sum()

type            0
zipcode         0
city            0
state           0
metro        1865
county          0
size_rank       0
2015-01         0
2015-02         0
2015-03         0
2015-04         0
2015-05         0
2015-06         0
2015-07         0
2015-08         0
2015-09         0
2015-10         0
2015-11         0
2015-12         0
dtype: int64

### Find average monthly and average 2015 home value

In [19]:
# Average monthly (2015) home value by state
monthly_state_estate = estate_df.groupby("state")[['2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12']].mean().reset_index()
monthly_state_estate.head()

,state,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12
0,AK,135346.088889,135547.622222,135574.155556,135693.266667,135963.733333,136288.488889,136598.688889,136888.511111,137111.000000,137221.644444,137350.911111,137723.066667
1,AL,53350.628099,53533.206612,53796.254132,54072.669421,54403.111570,54769.301653,55105.561983,55236.572314,55259.272727,55322.429752,55452.479339,55508.694215
2,AR,49468.953069,49411.888087,49432.259928,49473.288809,49572.725632,49800.032491,50048.324910,50231.501805,50403.220217,50583.014440,50750.458484,50888.776173
3,AZ,103289.851380,103624.670913,103978.430998,104349.048832,104714.677282,105032.380042,105476.273885,106122.972399,106790.976645,107330.849257,107795.505308,108236.384289
4,CA,262225.097831,264422.132214,266808.270978,268944.668850,270744.203029,272668.735571,274884.261973,277063.355710,278738.983627,280389.786328,282189.940647,283864.917315


In [20]:
# Average monthly (2015) home value by city
monthly_city_estate = estate_df.groupby("city")[['2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12']].mean().reset_index()
monthly_city_estate.head()

,city,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12
0,Abbeville,35691.600,35611.000,35528.400,35108.800,34592.000,34658.400,34930.60,35274.00,35613.000,35585.400,35418.800,35672.600
1,Aberdeen,62683.750,62384.625,62199.125,62138.125,62238.000,62387.125,62659.25,63108.75,63621.875,63949.500,64374.750,64811.375
2,Abernathy,34725.500,34722.000,34822.000,34818.500,34574.000,34474.000,34429.50,34381.00,34634.500,35086.500,35484.000,36030.000
3,Abilene,20891.625,20903.750,21003.125,21117.125,21359.375,21701.250,22018.50,22171.50,22348.000,22623.125,22869.625,22878.125
4,Abingdon,131838.750,131988.500,132813.750,134114.000,135214.000,135764.750,136342.25,136820.00,136996.250,137171.500,138245.500,139068.750


In [21]:
# Average 2015 home value
estate_df["value_average"] = estate_df.loc[:, ['2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12']].mean(axis=1)
estate_df.head()

C:\Users\zheng\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,type,zipcode,city,state,metro,county,size_rank,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,value_average
0,ZHVI,10025,New York,NY,New York,New York,1,908300,908600,904200,899400,901600,911500,926600,948100,970000,979800,983700,993700,936291.666667
1,ZHVI,60657,Chicago,IL,Chicago,Cook,2,308800,306700,306600,308800,310200,310000,310400,310000,310300,311800,312500,311600,309808.333333
2,ZHVI,60614,Chicago,IL,Chicago,Cook,3,368700,363600,357600,358000,359900,359300,359100,362000,368700,379500,386800,390300,367791.666667
3,ZHVI,79936,El Paso,TX,El Paso,El Paso,4,111700,111700,111600,111500,111500,111500,111700,111800,111800,111800,111500,111200,111608.333333
4,ZHVI,60640,Chicago,IL,Chicago,Cook,5,192200,191600,192700,195700,197200,195700,193500,192600,193400,195200,195900,195900,194300.000000


In [22]:
clean_estate = pd.pivot_table(estate_df, values = 'value_average', index=['zipcode'], columns = 'type').reset_index()
clean_estate = pd.merge(clean_estate, zipcode_to_city, on="zipcode", how="inner")
clean_estate.head(3)


,zipcode,ZHVI,ZRI,city,state
0,1001,181625.000000,1485.583333,Agawam,MA
1,1002,304000.000000,1989.750000,Amherst,MA
2,1005,192033.333333,1356.916667,Barre,MA


In [23]:
clean_estate = clean_estate.pivot_table(index=['city','state'], 
                                       values=['ZHVI', 'ZRI'], 
                                       aggfunc = {
                                           'ZHVI': np.mean,
                                           'ZRI': np.mean})

clean_estate.reset_index(inplace = True)
clean_estate.head(3)

,city,state,ZHVI,ZRI
0,Abbeville,AL,NaN,906.333333
1,Abbeville,GA,50866.666667,NaN
2,Abbeville,LA,NaN,1223.500000


In [24]:
clean_estate.shape

(12518, 4)

### Merge dataset:

In [25]:
# check the dataset want to merge:
# change city to lower case:
clean_listing['city'] = clean_listing['city'].apply(lambda x: x.lower())
clean_demographics['city'] = clean_demographics['city'].apply(lambda x: x.lower())
clean_estate['city'] = clean_estate['city'].apply(lambda x: x.lower())
clean_demographics.head(5)

,city,state,mean_household_income,median_household_income,total_population
0,aaronsburg,PA,57441.0,53000.0,1019
1,abbeville,AL,45482.0,33944.0,6257
2,abbeville,GA,40817.0,29200.0,4635
3,abbeville,LA,57431.0,42909.0,25707
4,abbeville,MS,81400.0,61563.0,2943


In [26]:
clean_listing.head()

,city,state,total_space_available,listing_count
0,new york,NY,346785.0,18903
1,brooklyn,NY,336442.0,16527
2,washington,DC,257492.0,7628
3,chicago,IL,209510.0,5130
4,denver,CO,73543.0,2421


In [27]:
clean_demographics.head()

,city,state,mean_household_income,median_household_income,total_population
0,aaronsburg,PA,57441.0,53000.0,1019
1,abbeville,AL,45482.0,33944.0,6257
2,abbeville,GA,40817.0,29200.0,4635
3,abbeville,LA,57431.0,42909.0,25707
4,abbeville,MS,81400.0,61563.0,2943


In [28]:
clean_demographics.shape

(25969, 5)

In [29]:
clean_estate.head()

,city,state,ZHVI,ZRI
0,abbeville,AL,NaN,906.333333
1,abbeville,GA,50866.666667,NaN
2,abbeville,LA,NaN,1223.500000
3,abbeville,MS,122475.000000,1063.750000
4,abbottstown,PA,172900.000000,1247.083333


In [30]:
clean_estate[clean_estate['city'] == 'queens']

,city,state,ZHVI,ZRI


In [31]:
#merge econ to listing:
#keep in mind, from econ we have state GDP, state income average, and state URate avg 
temp_merge_1 = clean_listing.merge(clean_econ, on='state', how='left')
#merge real estate to listing:
temp_merge_2 = temp_merge_1.merge(clean_estate, on = ['city','state'], how='left')
#merge demographic to listing:
final_table = temp_merge_2.merge(clean_demographics, on = ['city', 'state'], how='left')
final_table.shape


(93, 12)

In [32]:
#Check null of each columns
print(final_table.isnull().sum())
final_table.tail(10)

city                       0
state                      0
total_space_available      0
listing_count              0
annual_GDP                 0
personal_income_average    0
ur_average                 0
ZHVI                       8
ZRI                        2
mean_household_income      0
median_household_income    0
total_population           0
dtype: int64


,city,state,total_space_available,listing_count,annual_GDP,personal_income_average,ur_average,ZHVI,ZRI,mean_household_income,median_household_income,total_population
83,takoma park,MD,60.0,3,1461425,3.361874e+08,5.072727,460608.333333,2313.000000,98090.000000,69721.000000,25867
84,evergreen park,IL,54.0,1,3107526,6.467891e+08,4.181818,159941.666667,1487.000000,79638.000000,64385.000000,19318
85,yonkers,NY,50.0,2,5734124,1.161414e+09,6.490909,383815.000000,2499.983333,77055.800000,59704.800000,176071
86,evanston,IL,39.0,2,3107526,6.467891e+08,4.181818,332672.222222,2024.222222,119608.000000,79134.333333,78082
87,watertown,MA,28.0,1,1939772,4.253525e+08,4.381818,461791.666667,2422.500000,105413.000000,87409.000000,32709
88,milton,MA,28.0,1,1939772,4.253525e+08,4.381818,553341.666667,2688.583333,143069.000000,116444.000000,26502
89,quincy,MA,26.0,1,1939772,4.253525e+08,4.381818,382908.333333,2291.361111,81890.666667,67870.000000,91050
90,oak lawn,IL,16.0,1,3107526,6.467891e+08,4.181818,162883.333333,1532.583333,69353.000000,56116.000000,55189
91,chestnut hill,MA,6.0,3,1939772,4.253525e+08,4.381818,855483.333333,3407.083333,178967.000000,109399.000000,23068
92,jersey city,NJ,0.0,1,2270952,5.370264e+08,3.400000,365326.388889,2328.569444,111202.428571,90533.714286,256939


In [33]:
#After investigating the data, (since it's sort by total_space_avaiable, and the missing value appears at the bottom of table) we decide to drop these rows
#delete na and clear target column with zero value.
final_table['ZHVI'].replace('', np.nan, inplace=True)
final_table.dropna(inplace=True)
final_table = final_table[final_table.total_space_available != 0]

In [34]:
#Check the dimension of the finaldata set and recheck na
print(final_table.shape)
final_table.isnull().sum()


(84, 12)


city                       0
state                      0
total_space_available      0
listing_count              0
annual_GDP                 0
personal_income_average    0
ur_average                 0
ZHVI                       0
ZRI                        0
mean_household_income      0
median_household_income    0
total_population           0
dtype: int64

In [35]:
final_table.head(30)

,city,state,total_space_available,listing_count,annual_GDP,personal_income_average,ur_average,ZHVI,ZRI,mean_household_income,median_household_income,total_population
0,new york,NY,346785.0,18903,5734124,1.161414e+09,6.490909,1.272732e+06,3897.236111,149315.866667,94798.577778,1586411
1,brooklyn,NY,336442.0,16527,5734124,1.161414e+09,6.490909,6.589633e+05,2334.423423,72558.297297,51469.189189,2550822
2,washington,DC,257492.0,7628,488582,4.927592e+07,4.845455,5.387349e+05,2672.238095,109793.695652,79608.260870,629974
3,chicago,IL,209510.0,5130,3107526,6.467891e+08,4.181818,2.063277e+05,1672.258929,81886.410714,57735.964286,2663653
4,denver,CO,73543.0,2421,1254994,2.777318e+08,3.827273,3.094294e+05,1903.177632,77030.421053,57185.157895,1012535
5,boston,MA,57066.0,1686,1939772,4.253525e+08,4.381818,7.494824e+05,2969.628788,134378.384615,80857.153846,144617
6,astoria,NY,23393.0,1101,5734124,1.161414e+09,6.490909,6.099625e+05,2469.479167,74157.250000,56530.250000,148171
7,bronx,NY,23314.0,649,5734124,1.161414e+09,6.490909,3.108969e+05,1800.338889,53051.840000,39554.880000,1409173
8,staten island,NY,11702.0,257,5734124,1.161414e+09,6.490909,4.023764e+05,2141.395833,87952.750000,70786.416667,463707
9,flushing,NY,9609.0,237,5734124,1.161414e+09,6.490909,4.794917e+05,2230.166667,68675.500000,51431.000000,214497


### Output dataset:

In [36]:
#export the data to csv.
final_table.to_csv('datasets/final data test 5.csv', index=False)

In [ ]:
melt_final_table = final_table.melt(id_vars=["state"], value_vars=['total_space_available', 'listing_count', 'annual_GDP',       'personal_income_average', 'ur_average', 'ZHVI', 'ZRI', 'mean_household_income', 'median_household_income', 'total_population'])

# Graph each column by state
fig, ax = plt.subplots(figsize=(15, 7))
sns.barplot(x='state', y='value', hue='variable', data=melt_final_table, ax=ax).set_yscale("log")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.title("Rental, Economic, and Real Estate Data per State")
sns.despine(fig)